In [1]:
import json

# Load data from the JSON file
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
    data = json.load(file)
# Preprocess the data

food_items = []
for item in data:
    food_id = item['food']['food_id']
    food_name = item['food']['food_name']
    food_type = item['food']['food_type']
    servings = item['food']['servings']['serving']

    # Process each serving entry
    for serving in servings:
        serving_description = serving['serving_description']
        calories = serving['calories']
        carbohydrates = serving['carbohydrate']
        fat = serving['fat']
        protein = serving['protein']

        # Add the processed data to the food_items list
        food_items.append({
            'food_id': food_id,
            'food_name': food_name,
            'food_type': food_type,
            'serving_description': serving_description,
            'calories': calories,
            'carbohydrates': carbohydrates,
            'fat': fat,
            'protein': protein
        })
# Print the first few food items
for item in food_items[:20]:
    print(item)


{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '1 kebab', 'calories': '620', 'carbohydrates': '77.10', 'fat': '16.63', 'protein': '37.87'}
{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '1 serving (390 g)', 'calories': '620', 'carbohydrates': '77.10', 'fat': '16.63', 'protein': '37.87'}
{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '100 g', 'calories': '159', 'carbohydrates': '19.77', 'fat': '4.27', 'protein': '9.71'}
{'food_id': '2655309', 'food_name': 'Kebab', 'food_type': 'Generic', 'serving_description': '1 oz', 'calories': '45', 'carbohydrates': '5.60', 'fat': '1.21', 'protein': '2.75'}
{'food_id': '58704864', 'food_name': 'Gyoza', 'food_type': 'Brand', 'serving_description': '4 gyoza', 'calories': '120', 'carbohydrates': '17.00', 'fat': '2.50', 'protein': '7.00'}
{'food_id': '68848666', 'food_name': 'Gyoza', 'food_type': 'Brand', 'serving_descrip

In [4]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load data from the food.json file
try:
    with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
        food_data = json.load(file)
except FileNotFoundError:
    print("File not found")
    exit()

# Load user preferences from the user.json file
try:
    with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_dataset.json', 'r') as file:
        user_data = json.load(file)
except FileNotFoundError:
    print("File not found")
    exit()

# Preprocess the food data
food_items = []
for item in food_data:
    food_id = item['food']['food_id']
    food_name = item['food']['food_name']
    # Add other relevant features from the food.json file to the food_items list
    food_items.append((food_id, food_name))

# Preprocess the user preferences
if isinstance(user_data, list):
    user_preferences = user_data[0]['listUserPreferences']
else:
    user_preferences = user_data['listUserPreferences']

gender = user_preferences['gender']
weight = float(user_preferences['weight'])
# Add other relevant user preferences from the user.json file

# Define the variables
user_vocab_size = 1000
item_vocab_size = 1000
embedding_dim = 32

# user_ids = [...]  # List of user IDs
# Open the user_ids.json file and load its contents into the user_ids array
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_ids.json', 'r') as f:
    user_ids = json.load(f)


# item_ids = [...]  # List of item IDs
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/food_ids.json', 'r') as f:
    food_id = json.load(f)


# target_values = [...]  # List of target values
target_values = []

for user_id, item_id in zip(user_ids, food_id):
    # Assuming user_data is a dictionary with user preferences
    user_preferences = user_data[user_id]
    # Function to get harmful diseases for a food item
    harmful_diseases = get_harmful_diseases(item_id)

    if any(disease in user_preferences['disease'] for disease in harmful_diseases):
        # Assign lower target value for harmful foods
        target_values.append(0.0)
    else:
        # Assign higher target value for preferred foods
        target_values.append(1.0)



# Split the data into train, validation, and test sets
train_size = int(0.7 * len(user_ids))
val_size = int(0.15 * len(user_ids))
test_size = len(user_ids) - train_size - val_size

train_user_ids = user_ids[:train_size]
train_item_ids = item_ids[:train_size]
train_targets = target_values[:train_size]

val_user_ids = user_ids[train_size:train_size+val_size]
val_item_ids = item_ids[train_size:train_size+val_size]
val_targets = target_values[train_size:train_size+val_size]

test_user_ids = user_ids[train_size+val_size:]
test_item_ids = item_ids[train_size+val_size:]
test_targets = target_values[train_size+val_size:]

# Convert the data to TensorFlow Datasets
train_data = tf.data.Dataset.from_tensor_slices(
    (train_user_ids, train_item_ids, train_targets))
val_data = tf.data.Dataset.from_tensor_slices(
    (val_user_ids, val_item_ids, val_targets))
test_data = tf.data.Dataset.from_tensor_slices(
    (test_user_ids, test_item_ids, test_targets))

# Shuffle and batch the datasets
batch_size = 32
train_data = train_data.shuffle(buffer_size=train_size).batch(batch_size)
val_data = val_data.shuffle(buffer_size=val_size).batch(batch_size)
test_data = test_data.batch(batch_size)

# Define the collaborative filtering model


class CollaborativeFilteringModel(tf.keras.Model):
    def __init__(self, user_vocab_size, embedding_dim):
        super().__init__()
        # Define the layers for collaborative filtering
        self.embedding = tf.keras.layers.Embedding(
            user_vocab_size, embedding_dim)

    def call(self, inputs):
        user_embeddings = self.embedding(inputs['user_id'])
        return user_embeddings

# Define the content-based model


class ContentBasedModel(tf.keras.Model):
    def __init__(self, item_vocab_size, embedding_dim):
        super().__init__()
        # Define the layers for content-based filtering
        self.embedding = tf.keras.layers.Embedding(
            item_vocab_size, embedding_dim)

    def call(self, inputs):
        item_embeddings = self.embedding(inputs['item_id'])
        return item_embeddings

# Define the hybrid model


class HybridModel(tfrs.Model):
    def __init__(self, user_vocab_size, item_vocab_size, embedding_dim):
        super().__init__()
        self.collaborative_model = CollaborativeFilteringModel(
            user_vocab_size, embedding_dim)
        self.content_based_model = ContentBasedModel(
            item_vocab_size, embedding_dim)
        self.dense_layer = tf.keras.layers.Dense(32, activation='relu')
        self.final_layer = tf.keras.layers.Dense(1)

    def call(self, inputs):
        user_embeddings = self.collaborative_model(inputs)
        item_embeddings = self.content_based_model(inputs)
        concatenated_embeddings = tf.concat(
            [user_embeddings, item_embeddings], axis=1)
        x = self.dense_layer(concatenated_embeddings)
        output = self.final_layer(x)
        return output


# Define the loss function and metrics
loss = tf.keras.losses.MeanSquaredError()
metrics = [tf.keras.metrics.RootMeanSquaredError()]

# Create an instance of the hybrid model
model = HybridModel(user_vocab_size, item_vocab_size, embedding_dim)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001), loss=loss, metrics=metrics)

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)

# Evaluate the model
results = model.evaluate(test_data)
print(f"Test RMSE: {results[1]}")

# Make recommendations
# User ID for which recommendations will be generated
user_inputs = {'user_id': tf.constant([user_ids])}
# Item IDs to consider for recommendations
item_inputs = {'item_id': tf.constant(item_ids)}
recommendations = model.predict((user_inputs, item_inputs))
print(recommendations)


ValueError: Attempt to convert a value (Ellipsis) with an unsupported type (<class 'ellipsis'>) to a Tensor.

In [1]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs


In [3]:
# Load data from the food.json file
try:
    with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
        food_data = json.load(file)
except FileNotFoundError:
    print("File not found")
    exit()

# Load user preferences from the user.json file
try:
    with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_dataset_new.json', 'r') as file:
        user_data = json.load(file)
except FileNotFoundError:
    print("File not found")
    exit()


In [4]:
# Define the function to get harmful diseases for a given food item
def get_harmful_diseases(food_id):
    harmful_diseases = []
    for item in food_data:
        if item['food']['food_id'] == food_id:
            harmful_diseases = item['food']['harmful_diseases']
            break
    return harmful_diseases


In [5]:
# Preprocess the food data
food_items = []
for item in food_data:
    food_id = item['food']['food_id']
    food_name = item['food']['food_name']
    # Add other relevant features from the food.json file to the food_items list
    food_items.append((food_id, food_name))


In [6]:
# Preprocess the user preferences
if isinstance(user_data, list):
    user_data = user_data[0]
user_preferences = user_data['listUserPreferences']
gender = user_preferences['gender']
weight = float(user_preferences['weight'])


In [7]:
# Define the variables
user_vocab_size = 1000
item_vocab_size = 1000
embedding_dim = 32


In [8]:
# user_ids = [...]  # List of user IDs
# Open the user_ids.json file and load its contents into the user_ids array
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_ids.json', 'r') as f:
    user_ids = json.load(f)

# item_ids = [...]  # List of item IDs
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/food_ids.json', 'r') as f:
    item_ids = json.load(f)['food_ids']

target_values = []  # List of target values

# Fill target values based on user preferences and harmful diseases
for user_id, item_id in zip(user_ids, item_ids):
    user_preference = user_data.get(str(user_id))
    if user_preference is None:
        # User preferences not found, assign lower target value
        target_values.append(0.0)
    else:
        user_preference = user_preference['listUserPreferences']
        harmful_diseases = get_harmful_diseases(item_id)

        if any(disease in user_preference['disease'] for disease in harmful_diseases):
            # Assign lower target value for harmful foods
            target_values.append(0.0)
        else:
            # Assign higher target value for preferred foods
            target_values.append(1.0)


In [9]:
# Split the data into train, validation, and test sets
train_size = int(0.7 * len(user_ids))
val_size = int(0.15 * len(user_ids))
test_size = len(user_ids) - train_size - val_size

train_user_ids = user_ids[:train_size]
train_item_ids = item_ids[:train_size]
train_targets = target_values[:train_size]

val_user_ids = user_ids[train_size:train_size+val_size]
val_item_ids = item_ids[train_size:train_size+val_size]
val_targets = target_values[train_size:train_size+val_size]

test_user_ids = user_ids[train_size+val_size:]
test_item_ids = item_ids[train_size+val_size:]
test_targets = target_values[train_size+val_size:]


In [10]:
print("train_user_ids size:", len(train_user_ids))
print("train_item_ids size:", len(train_item_ids))
print("train_targets size:", len(train_targets))

print("val_user_ids size:", len(val_user_ids))
print("val_item_ids size:", len(val_item_ids))
print("val_targets size:", len(val_targets))

print("test_user_ids size:", len(test_user_ids))
print("test_item_ids size:", len(test_item_ids))
print("test_targets size:", len(test_targets))


train_user_ids size: 350
train_item_ids size: 265
train_targets size: 265
val_user_ids size: 75
val_item_ids size: 0
val_targets size: 0
test_user_ids size: 75
test_item_ids size: 0
test_targets size: 0


In [11]:
# Convert the data to TensorFlow Datasets
train_data = tf.data.Dataset.from_tensor_slices(
    ({"user_id": train_user_ids, "item_id": train_item_ids}, train_targets)
)
val_data = tf.data.Dataset.from_tensor_slices(
    ({"user_id": val_user_ids, "item_id": val_item_ids}, val_targets)
)
test_data = tf.data.Dataset.from_tensor_slices(
    ({"user_id": test_user_ids, "item_id": test_item_ids}, test_targets)
)


ValueError: Dimensions 265 and 350 are not compatible

In [12]:
# Build the model
class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.user_embedding = tf.keras.layers.Embedding(
            user_vocab_size, embedding_dim)

    def call(self, inputs):
        return self.user_embedding(inputs["user_id"])


class ItemModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.item_embedding = tf.keras.layers.Embedding(
            item_vocab_size, embedding_dim)

    def call(self, inputs):
        return self.item_embedding(inputs["item_id"])


class RecommenderModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.user_model = UserModel()
        self.item_model = ItemModel()
        self.task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features)
        item_embeddings = self.item_model(features)
        return self.task(user_embeddings, item_embeddings)

In [13]:
# Create an instance of the RecommenderModel
recommender_model = RecommenderModel()

# Compile the model
recommender_model.compile(
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Train the model
recommender_model.fit(train_data.shuffle(1000).batch(32), epochs=10)

# Evaluate the model
eval_results = recommender_model.evaluate(val_data.batch(32), return_dict=True)
print(f"Validation RMSE: {eval_results['root_mean_squared_error']}")

# Make predictions
predictions = recommender_model.predict(test_data.batch(32))

NameError: name 'train_data' is not defined

In [14]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

# Load data from the food.json file
try:
    with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
        food_data = json.load(file)
except FileNotFoundError:
    print("File not found")
    exit()

# Load user preferences from the user.json file
try:
    with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_dataset.json', 'r') as file:
        user_data = json.load(file)
except FileNotFoundError:
    print("File not found")
    exit()

# Preprocess the food data
food_items = []
for item in food_data:
    food_id = item['food']['food_id']
    food_name = item['food']['food_name']
    # Add other relevant features from the food.json file to the food_items list
    food_items.append((food_id, food_name))

# Preprocess the user preferences
if isinstance(user_data, list):
    user_preferences = user_data[0]['listUserPreferences']
else:
    user_preferences = user_data['listUserPreferences']

gender = user_preferences['gender']
weight = float(user_preferences['weight'])
# Add other relevant user preferences from the user.json file

# Define the variables
user_vocab_size = 1000
item_vocab_size = 1000
embedding_dim = 32

# Load user and item IDs from files
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_ids.json', 'r') as f:
    user_ids = json.load(f)

with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/food_ids.json', 'r') as f:
    food_ids = json.load(f)

# Define target values based on user preferences and harmful diseases
target_values = []
for user_id, item_id in zip(user_ids, food_ids):
    user_preferences = user_data[user_id]['listUserPreferences']
    harmful_diseases = get_harmful_diseases(item_id)

    if any(disease in user_preferences['disease'] for disease in harmful_diseases):
        # Assign lower target value for harmful foods
        target_values.append(0.0)
    else:
        # Assign higher target value for preferred foods
        target_values.append(1.0)

# Split the data into train, validation, and test sets
train_size = int(0.7 * len(user_ids))
val_size = int(0.15 * len(user_ids))
test_size = len(user_ids) - train_size - val_size

train_user_ids = user_ids[:train_size]
train_item_ids = food_ids[:train_size]
train_targets = target_values[:train_size]

val_user_ids = user_ids[train_size:train_size+val_size]
val_item_ids = food_ids[train_size:train_size+val_size]
val_targets = target_values[train_size:train_size+val_size]

test_user_ids = user_ids[train_size+val_size:]
test_item_ids = food_ids[train_size+val_size:]
test_targets = target_values[train_size+val_size:]

# Convert the data to TensorFlow Datasets
train_data = tf.data.Dataset.from_tensor_slices(
    ({"user_id": train_user_ids, "item_id": train_item_ids}, train_targets))
val_data = tf.data.Dataset.from_tensor_slices(
    ({"user_id": val_user_ids, "item_id": val_item_ids}, val_targets))
test_data = tf.data.Dataset.from_tensor_slices(
    ({"user_id": test_user_ids, "item_id": test_item_ids}, test_targets))

# Shuffle and batch the datasets
batch_size = 32
train_data = train_data.shuffle(buffer_size=train_size).batch(batch_size)
val_data = val_data.shuffle(buffer_size=val_size).batch(batch_size)
test_data = test_data.batch(batch_size)

# Define the collaborative filtering model


class CollaborativeFilteringModel(tf.keras.Model):
    def __init__(self, user_vocab_size, embedding_dim):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            user_vocab_size, embedding_dim)

    def call(self, inputs):
        user_embeddings = self.embedding(inputs['user_id'])
        return user_embeddings

# Define the content-based model


class ContentBasedModel(tf.keras.Model):
    def __init__(self, item_vocab_size, embedding_dim):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(
            item_vocab_size, embedding_dim)

    def call(self, inputs):
        item_embeddings = self.embedding(inputs['item_id'])
        return item_embeddings

# Define the hybrid model


class HybridModel(tfrs.Model):
    def __init__(self, user_vocab_size, item_vocab_size, embedding_dim):
        super().__init__()
        self.collaborative_model = CollaborativeFilteringModel(
            user_vocab_size, embedding_dim)
        self.content_based_model = ContentBasedModel(
            item_vocab_size, embedding_dim)
        self.dense_layer = tf.keras.layers.Dense(32, activation='relu')
        self.final_layer = tf.keras.layers.Dense(1)

    def call(self, inputs):
        user_embeddings = self.collaborative_model(inputs)
        item_embeddings = self.content_based_model(inputs)
        concatenated_embeddings = tf.concat(
            [user_embeddings, item_embeddings], axis=1)
        x = self.dense_layer(concatenated_embeddings)
        output = self.final_layer(x)
        return output


# Create an instance of the hybrid model
model = HybridModel(user_vocab_size, item_vocab_size, embedding_dim)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

# Train the model
model.fit(train_data, validation_data=val_data, epochs=10)

# Evaluate the model
results = model.evaluate(test_data)
print(f"Test RMSE: {results[1]}")

# Make recommendations
# User IDs for which recommendations will be generated
user_inputs = {'user_id': np.array(user_ids)}
# Item IDs to consider for recommendations
item_inputs = {'item_id': np.array(food_ids)}
recommendations = model.predict((user_inputs, item_inputs))
print(recommendations)


TypeError: list indices must be integers or slices, not str

In [9]:
import json
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import random

# Load the datasets
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_dataset.json', 'r') as file:
    user_data = json.load(file)  # Load user.json dataset
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/output.json', 'r') as file:
    food_data = json.load(file)  # Load food.json dataset
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/user_ids.json', 'r') as f:
    user_ids = json.load(f)  # Load user_ids.json dataset
with open('C:/Users/Ananda/OneDrive/repos/NutriPal-ML/results/food_ids.json', 'r') as f:
    food_ids = json.load(f)  # Load food_ids.json dataset

# Define the model inputs and features


class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Define the user features
        self.user_embedding = tf.keras.layers.Embedding(
            len(user_ids), output_dim=32
        )

    def call(self, inputs):
        user_embedding = self.user_embedding(inputs["user_id"])
        return user_embedding


class FoodModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # Define the food features
        self.food_embedding = tf.keras.layers.Embedding(
            len(food_ids), output_dim=32
        )

    def call(self, inputs):
        food_embedding = self.food_embedding(inputs["food_id"])
        return food_embedding

# Define the loss and metrics


class RecommenderModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        # Define the user and food models
        self.user_model = UserModel()
        self.food_model = FoodModel()

        # Define the retrieval task
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=self.food_model
            )
        )

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features)
        positive_food_embeddings = self.food_model(features)

        # Sample negative food examples
        negative_food_ids = tf.random.shuffle(food_ids_tensor)[:100]
        negative_food_embeddings = self.food_model(
            {"food_id": negative_food_ids})

        # Concatenate positive and negative examples
        food_embeddings = tf.concat(
            [positive_food_embeddings, negative_food_embeddings], axis=0)

        return self.task(user_embeddings, food_embeddings)


# Convert food_ids to TensorFlow tensor
food_ids_tensor = tf.constant(food_ids)

# Instantiate the model and compile it
model = RecommenderModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Prepare the data for training


def prepare_data():
    for user in user_data:
        user_id = user["id_user"]
        user_food_preferences = user["listUserPreferences"]["favoriteFood"]
        user_diseases = user["listUserPreferences"]["disease"]

        for food in food_data:
            food_id = food["food"]["food_id"]
            food_name = food["food"]["food_name"]

            # Filter foods based on user's disease
            is_safe_food = True
            for disease in user_diseases:
                if disease in food_name.lower():
                    is_safe_food = False
                    break

            if is_safe_food:
                # Create positive example (user, food)
                yield {
                    "user_id": user_ids.index(user_id),
                    "food_id": food_ids.index(food_id)
                }

                # Create negative examples by randomly sampling foods
                for _ in range(5):
                    random_food_id = random.choice(food_ids)
                    yield {
                        "user_id": user_ids.index(user_id),
                        "food_id": food_ids.index(random_food_id)
                    }


train_data = tf.data.Dataset.from_generator(prepare_data, output_signature=(
    {"user_id": tf.TensorSpec(shape=(), dtype=tf.int32),
     "food_id": tf.TensorSpec(shape=(), dtype=tf.int32)}
))

model.fit(train_data.batch(4096), epochs=10)


Epoch 1/10


AttributeError: in user code:

    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\tensorflow_recommenders\models\base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "C:\Users\Ananda\AppData\Local\Temp\ipykernel_5388\4247999888.py", line 75, in compute_loss
        return self.task(user_embeddings, food_embeddings)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback